#### TestMy

注意：
+ 一个城市对应多条返回结果
+ 检索时应将非英文字符转化为英文字母，如Montréal
+ 去掉城市中County部分，如Fairfax County

In [ ]:
import bs4
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random

def get_testMy(url, city):
    # 初始化浏览器自动化工具
    driver = webdriver.Chrome()    
    driver.get(url)    # 获取页面
    time.sleep(4)    # 设置休眠时间
    
    # 创建数据表存储结果
    data = pd.DataFrame(columns=['city', 'City', 'Name', 'State', 'Country', 'Average Up', 'Average Down', 'Count'])
    # 获取城市列表
    city_list = city['city']
    for i in range(len(city_list)):
        # 获取城市名称
        ct = data.iloc[i, 0]
        # 替换特殊字符
        if 'é' in ct:
            ct = ct.replace('é', 'e')
        if 'è' in ct:
            ct = ct.replace('è', 'e')
        if 'ó' in ct:
            ct = ct.replace('ó', 'o')
        if 'ö' in ct:
            ct = ct.replace('ö', 'o')
        if 'ō' in ct:
            ct = ct.replace('ō', 'o')
        if 'ā' in ct:
            ct = ct.replace('ā', 'a')
        if 'H̱' in ct:
            ct = ct.replace('H̱', 'H')
        if 'ü' in ct:
            ct = ct.replace('ü', 'u')
        # 获取该输入框的Xpath
        input_element = driver.find_element(By.XPATH,'(//input[@class="form-control"])')   
        input_element.click()        # 点击输入框
        input_element.clear()        # 清除该输入框中的原本内容
        input_element.send_keys(ct)     # 向该输入框中添加搜索词
        input_element.send_keys(Keys.ENTER)     # 回车搜索
        # 随机休眠
        time.sleep(random.randint(1, 5))
        
        # 解析页面
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            # 获取全部tr标签
            for tr in soup.find('tbody').children:
                # 若为标题则跳过
                if tr.find('th'):
                    continue
                # 存储第一个返回结果
                elif tr.find('td'):
                    tds = tr.find_all('td')
                    if tds[2].find('a') and tds[3].find('a'):    # 若同时包含省份、国家
                        value = [ct, tds[0].a.text, tds[0].span.text, tds[2].a.text, tds[3].a.text, tds[4].div.text, tds[5].div.text, tds[6].div.text]
                    elif tds[3].find('a'):    # 若仅包含国家
                        value = [ct, tds[0].a.text, tds[0].span.text, '', tds[3].a.text, tds[4].div.text, tds[5].div.text, tds[6].div.text]
                    else:    # 若均不包含
                        value = [ct, tds[0].a.text, tds[0].span.text, '', '', tds[4].div.text, tds[5].div.text, tds[6].div.text]
                    break
        # 若查询结果为空
        except:
            value = [ct, '', '', '', '', '', '', '']
        # 将结果写入数据表
        data.loc[len(data.index)] = value
        # 打印结果
        print(value)
    # 将结果写入excel    
    data.to_excel('TestMy.xlsx', index=False)

city = pd.read_excel('文件3：GIHI2023附加子城市列表.xlsx')
url = 'https://testmy.net/city'
get_testMy(url, city)

['Montréal', '', '', '', '', '', '', '']
['Laval', 'Laval', 'laval_qc', 'QC', 'CA', '135 Mbps', '33.7 Mbps', '44761']
['Longueuil', 'Longueuil', 'longueuil_qc', 'QC', 'CA', '128.8 Mbps', '25.7 Mbps', '25956']
['Toronto', 'Toronto', 'toronto_on', 'ON', 'CA', '115.7 Mbps', '20.9 Mbps', '45126']
['Mississauga', 'Mississauga', 'mississauga_on', 'ON', 'CA', '194.3 Mbps', '66.6 Mbps', '45003']
['Brampton', 'Brampton', 'brampton_on', 'ON', 'CA', '123.7 Mbps', '38.4 Mbps', '45011']
['Markham', 'Markham', 'markham_on', 'ON', 'CA', '169.1 Mbps', '22.9 Mbps', '45001']
['Vaughan', 'Vaughan', 'vaughan_on', 'ON', 'CA', '89.4 Mbps', '15.5 Mbps', '9300']
['Richmond Hill', 'Richmond Hill', 'richmond_hill_on', 'ON', 'CA', '197.2 Mbps', '24 Mbps', '45014']
['Oakville', 'Oakville', 'oakville_on', 'ON', 'CA', '146.6 Mbps', '24.2 Mbps', '45001']
['Burlington', 'Burlington', 'burlington_on', 'ON', 'CA', '95.6 Mbps', '14.1 Mbps', '45003']
['Oshawa', 'Oshawa', 'oshawa_on', 'ON', 'CA', '173.6 Mbps', '36.5 Mbps'

['Pasadena', 'Pasadena', 'pasadena_ca', 'CA', 'US', '139.6 Mbps', '20.2 Mbps', '45000']
['Orange', '', '', '', '', '', '', '']
['Fullerton', 'Fullerton', 'fullerton_ca', 'CA', 'US', '125.1 Mbps', '24.5 Mbps', '23122']
['El Monte', 'San Jose del Monte', 'san_jose_del_mont', 'BUL', 'PH', '62.8 Mbps', '39.4 Mbps', '39159']
['Downey', 'Downey', 'downey_ca', 'CA', 'US', '90.4 Mbps', '8.8 Mbps', '15589']
['Costa Mesa', 'Costa Mesa', 'costa_mesa_ca', 'CA', 'US', '114.1 Mbps', '27.4 Mbps', '45000']
['Inglewood', 'Inglewood', 'inglewood_ca', 'CA', 'US', '93.1 Mbps', '12.5 Mbps', '10137']
['Burbank', 'Burbank', 'burbank_ca', 'CA', 'US', '145.8 Mbps', '21.1 Mbps', '45002']
['Culver City', 'Culver City', 'culver_city_ca', 'CA', 'US', '100.4 Mbps', '27.7 Mbps', '16100']
['El Segundo', 'El Segundo', 'el_segundo_ca', 'CA', 'US', '119 Mbps', '20.1 Mbps', '18544']
['Gardiner', 'Gardiner', 'gardiner_mt', 'MT', 'US', '13.4 Mbps', '6.1 Mbps', '4261']
['Hawthorne', 'Hawthorne', 'hawthorne_ca', 'CA', 'US', 

['Campbell', 'Campbell', 'campbell_ca', 'CA', 'US', '121 Mbps', '23.6 Mbps', '17438']
['Cupertino', 'Cupertino', 'cupertino_ca', 'CA', 'US', '134.8 Mbps', '29.2 Mbps', '33603']
['Emeryville', 'Emeryville', 'emeryville_ca', 'CA', 'US', '55.3 Mbps', '15.1 Mbps', '12903']
['Foster City', 'Foster City', 'foster_city_ca', 'CA', 'US', '17.6 Mbps', '5 Mbps', '2555']
['Half Moon Bay', 'Half Moon Bay', 'half_moon_bay_ca', 'CA', 'US', '57.3 Mbps', '7.1 Mbps', '6632']
['Los Altos', 'Los Altos', 'los_altos_ca', 'CA', 'US', '81.8 Mbps', '11.8 Mbps', '10962']
['Marin County', '', '', '', '', '', '', '']
['Milpitas', 'Milpitas', 'milpitas_ca', 'CA', 'US', '115.4 Mbps', '29 Mbps', '27594']
['Moffett Field', '', '', '', '', '', '', '']
['Mountain View', 'Mountain View', 'mountain_view_ca', 'CA', 'US', '94 Mbps', '14.9 Mbps', '45002']
['Palo Alto', 'Palo Alto', 'palo_alto_ca', 'CA', 'US', '110.4 Mbps', '21.4 Mbps', '42741']
['Pleasanton', 'Pleasanton', 'pleasanton_ca', 'CA', 'US', '89.8 Mbps', '15 Mbps'

['Jiangmen', '', '', '', '', '', '', '']
['Zhaoqing', '', '', '', '', '', '', '']
['Hangzhou', '', '', '', '', '', '', '']
['Harbin', '', '', '', '', '', '', '']
['Hefei', '', '', '', '', '', '', '']
['Jinan', '', '', '', '', '', '', '']
['Lanzhou', '', '', '', '', '', '', '']
['Nanjing', '', '', '', '', '', '', '']
['Qingdao', 'Qingdao', 'qingdao_sd', 'SD', 'CN', '38.8 Mbps', '10.8 Mbps', '226']
['Shanghai', '', '', '', '', '', '', '']
['Suzhou', 'Suzhou', 'suzhou_js', 'JS', 'CN', '23 Mbps', '7.4 Mbps', '4276']
['Taipei', 'Taipei', 'taipei_tpe', 'TPE', 'TW', '78.2 Mbps', '25.2 Mbps', '45016']
['Tianjin', '', '', '', '', '', '', '']
['Wuhan', 'Wuhan', 'wuhan_hb', 'HB', 'CN', '36.4 Mbps', '27.9 Mbps', '12572']
['Xiamen', '', '', '', '', '', '', '']
["Xi'an", '', '', '', '', '', '', '']
['Zhengzhou', '', '', '', '', '', '', '']
['Bengaluru/Bangalore', '', '', '', '', '', '', '']
['Delhi', 'Delhi', 'delhi_dl', 'DL', 'IN', '71.1 Mbps', '29.1 Mbps', '45240']
['Faridabad', '', '', '', '', ''